In [1]:
import itertools
import gym
import torch
import torch.nn as nn

In [2]:
import _init_paths

add code root path (with `rllib`).


In [3]:
from rllib.ddqn import DoubleDQN
from rllib.misc import total_weights

In [4]:
env = gym.make("CartPole-v1")
action_n = env.action_space.n
observation_shape = env.observation_space.shape

# net = nn.Sequential(nn.Linear(observation_shape[0],64),nn.BatchNorm1d(64),nn.ReLU(),
#                     nn.Linear(64,32),nn.BatchNorm1d(32),nn.ReLU(),
#                     nn.Linear(32,action_n))
net = nn.Sequential(nn.Linear(observation_shape[0],64),nn.ReLU(),
                    nn.Linear(64,32),nn.ReLU(),
                    nn.Linear(32,action_n))
print(net)
print("Model size:", total_weights(net))
agent = DoubleDQN(model=net, action_n=action_n, 
                  gamma=0.6, memory_size=10000,
                  learning_rate=1.e-4, batch_size=256)

Sequential(
  (0): Linear(in_features=4, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=32, bias=True)
  (3): ReLU()
  (4): Linear(in_features=32, out_features=2, bias=True)
)
Model size: 2466


In [5]:
running_reward = 0
for episode in range(2000):
    obs = env.reset()
    total_reward = 0
    for _ in range(10000): # not exceed 10000 episodes
        action = agent.select_action(obs,episode)
        next_obs, reward, done, _  = env.step(action)
#         env.render()
        total_reward+=reward
        if done:
            agent.memorize(obs, action, None, reward)
            agent.update_target_model()
            break
        else:
            agent.memorize(obs, action, next_obs, reward)
            obs = next_obs
        train_loss = agent.step()
    count_gamma = 0.5
    running_reward = count_gamma*running_reward+(1-count_gamma)*total_reward
#         print(l)
    print(episode, total_reward, running_reward, train_loss)
    if running_reward>env.spec.reward_threshold:
        break
print("Finished: %s@%s" %(running_reward,episode))

0 56.0 28.0 None
1 14.0 21.0 None
2 16.0 18.5 None
3 25.0 21.75 None
4 43.0 32.375 None
5 29.0 30.6875 None
6 36.0 33.34375 None
7 18.0 25.671875 None
8 14.0 19.8359375 None
9 21.0 20.41796875 0.4889112114906311
10 24.0 22.208984375 0.46463796496391296
11 14.0 18.1044921875 0.46137139201164246
12 34.0 26.05224609375 0.4162195324897766
13 16.0 21.026123046875 0.4196425676345825
14 24.0 22.5130615234375 0.39807653427124023
15 14.0 18.25653076171875 0.3909388780593872
16 12.0 15.128265380859375 0.37951889634132385
17 16.0 15.564132690429688 0.3676362633705139
18 15.0 15.282066345214844 0.3507424592971802
19 14.0 14.641033172607422 0.3405704200267792
20 22.0 18.32051658630371 0.3143009841442108
21 9.0 13.660258293151855 0.3243410289287567
22 26.0 19.830129146575928 0.28166672587394714
23 15.0 17.415064573287964 0.28213414549827576
24 19.0 18.207532286643982 0.25549814105033875
25 26.0 22.10376614332199 0.2358492612838745
26 18.0 20.051883071660995 0.23151396214962006
27 16.0 18.02594153583

KeyboardInterrupt: 

In [6]:
torch.save(net.state_dict(), "double_dqn.pth")

In [7]:
for i in range(5):
    obs = env.reset()
    total_reward=0
    for _ in range(10000): # not exceed 10000 episodes
        action = agent.play(obs)
        obs, reward, done, _ = env.step(action)
        env.render()
        total_reward+=reward
        if done:
             break
    print("Encore: %s"%total_reward)

Encore: 194.0
Encore: 191.0
Encore: 191.0
Encore: 208.0
Encore: 205.0


In [8]:
# dqn seems to be parameter-sensitive